In [1]:
# lire les documents depuis le répertoire et les stocker comme document avec des méta data
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader('./data').load_data()

In [2]:
#génerer des noeuds à partir des documents
from llama_index.node_parser import SimpleNodeParser

parser = SimpleNodeParser()

nodes = parser.get_nodes_from_documents(documents)

In [3]:
# define prompt
from llama_index import Prompt

template = (
    "Le contexte étant le suivant:. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Veuillez répondre à la question compte tenu ces informations: {query_str}\n"
)
qa_template = Prompt(template)

In [6]:
# générer l'index suivant les noeuds dépendamment de notre besoin
# nous allons générer un index de vecteur adéquat pour les Q&A
# nous allons générer un index avec comme model d'embedding OpenAI 

import os
import openai
from llama_index.embeddings import OpenAIEmbedding
from llama_index import ServiceContext, set_global_service_context, VectorStoreIndex, LLMPredictor, StorageContext
from langchain import OpenAI

import torch
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt

from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from sentence_transformers import models, SentenceTransformer
from llama_index.embeddings import LangchainEmbedding


index_name = "./saved_index"

"""
#enregistrer notre openAi api key coome variable global
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
openai.api_key = os.environ["OPENAI_API_KEY"]

embed_model = OpenAIEmbedding(embed_batch_size=10)

service_context = ServiceContext.from_defaults(embed_model=embed_model)
# mettre ce service context celui par défaut
set_global_service_context(service_context)

# build index
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context, show_progress=True
)
"""
model="sentence-transformers/all-mpnet-base-v2"
langchain_embedding=HuggingFaceEmbeddings(model_name=model)

embed_model = LangchainEmbedding(langchain_embedding=langchain_embedding)


service_context = ServiceContext.from_defaults(embed_model=embed_model)

# mettre ce service context celui par défaut
set_global_service_context(service_context)

# build index
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context, show_progress=True,summary_template=qa_template
)

index.storage_context.persist(index_name)


Generating embeddings: 100%|██████████| 46/46 [00:38<00:00,  1.20it/s]


In [5]:
query_engine = index.as_query_engine()
response = query_engine.query("Bonjour")
response

AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.